In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import nltk


In [2]:
import re
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [3]:
import spacy
import pyLDAvis
import pyLDAvis.gensim

In [32]:
df = pd.read_csv("all-pages-to-tag.csv")

In [33]:
df.head()

,File,Page,Text
0,81363.pdf,2,Toronto and Region Conservation TRCA has beco...
1,81363.pdf,3,An Overview of Getting to Carbon Neutral: A Gu...
2,81363.pdf,4,Getting to Carbon Neutral does not dictate wh...
3,81363.pdf,5,Tens steps to carbon neutrality Getting to Ca...
4,81363.pdf,6,fired generating stations emit large amounts ...


In [6]:
stopwords = nltk.corpus.stopwords.words('english')
stopwords.extend(['may'])


In [7]:
#tokenization

def sent_to_words(raw_text):
    for text in raw_text:
        yield(gensim.utils.simple_preprocess(str(text), deacc=True)) #deacc=True removes punctuations

words = list(sent_to_words(df['Text']))

print(words[:1])

[['toronto', 'and', 'region', 'conservation', 'trca', 'has', 'become', 'trusted', 'source', 'of', 'information', 'on', 'climate', 'change', 'adaptation', 'and', 'mitigation', 'getting', 'to', 'carbon', 'neutral', 'is', 'one', 'in', 'series', 'of', 'trca', 'publications', 'designed', 'to', 'encourage', 'environmental', 'projects', 'and', 'partnerships', 'for', 'livable', 'sustainable', 'future', 'for', 'more', 'information', 'on', 'combating', 'climate', 'change', 'and', 'the', 'ways', 'communities', 'companies', 'and', 'individuals', 'can', 'reduce', 'their', 'carbon', 'footprints', 'visit', 'our', 'website', 'at', 'www', 'trca', 'on', 'ca', 'written', 'by', 'william', 'glenn', 'wm', 'glenn', 'environmental', 'consultants', 'based', 'on', 'the', 'report', 'getting', 'to', 'carbon', 'neutral', 'guide', 'for', 'canadian', 'municipalities', 'by', 'the', 'sustainable', 'infrastructure', 'group', 'at', 'the', 'university', 'of', 'toronto', 'for', 'toronto', 'and', 'region', 'conservation', 

In [8]:
# Bigram and trigram models

bigram = gensim.models.Phrases(words, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[words], threshold=100)  

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

print(trigram_mod[bigram_mod[words[0]]])

['toronto', 'and', 'region', 'conservation', 'trca', 'has', 'become', 'trusted', 'source', 'of', 'information', 'on', 'climate', 'change', 'adaptation', 'and', 'mitigation', 'getting', 'to', 'carbon_neutral', 'is', 'one', 'in', 'series', 'of', 'trca', 'publications', 'designed', 'to', 'encourage', 'environmental', 'projects', 'and', 'partnerships', 'for', 'livable', 'sustainable', 'future', 'for', 'more', 'information', 'on', 'combating', 'climate', 'change', 'and', 'the', 'ways', 'communities', 'companies', 'and', 'individuals', 'can', 'reduce', 'their', 'carbon', 'footprints', 'visit', 'our', 'website', 'at', 'www', 'trca', 'on', 'ca', 'written', 'by', 'william', 'glenn', 'wm', 'glenn', 'environmental', 'consultants', 'based', 'on', 'the', 'report', 'getting', 'to', 'carbon_neutral', 'guide', 'for', 'canadian', 'municipalities', 'by', 'the', 'sustainable', 'infrastructure', 'group', 'at', 'the', 'university', 'of', 'toronto', 'for', 'toronto', 'and', 'region', 'conservation', 'fundin

In [9]:
# Stopwords, bigrams, trigrams and lemmatization

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stopwords] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):    
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [10]:
# Remove Stop Words
words_nostops = remove_stopwords(words)

# Form Bigrams
words_bigrams = make_bigrams(words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
words_lemmatized = lemmatization(words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(words_lemmatized[:1])

C:\Users\user\Anaconda3\lib\site-packages\msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)
C:\Users\user\Anaconda3\lib\site-packages\msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)


[['region', 'conservation', 'trca', 'become', 'trusted', 'source', 'information', 'climate', 'change', 'adaptation', 'mitigation', 'get', 'carbon_neutral', 'series', 'trca', 'publication', 'design', 'encourage', 'environmental', 'project', 'partnership', 'livable', 'sustainable', 'future', 'information', 'combat', 'climate', 'change', 'way', 'community', 'company', 'individual', 'reduce', 'carbon', 'footprint', 'visit', 'website', 'www', 'trca', 'ca', 'write', 'william', 'glenn', 'glenn', 'environmental', 'consultant', 'base', 'report', 'get', 'carbon_neutral', 'guide', 'canadian', 'municipality', 'sustainable', 'infrastructure', 'group', 'university', 'toronto', 'region', 'conservation', 'funding', 'provide', 'region', 'peel', 'york', 'region', 'city', 'right', 'reserve']]


In [11]:
# Create dictionary
id2word = corpora.Dictionary(words_lemmatized)

# Create corpus
texts = words_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 2), (7, 2), (8, 1), (9, 2), (10, 1), (11, 1), (12, 1), (13, 2), (14, 1), (15, 1), (16, 1), (17, 2), (18, 1), (19, 1), (20, 1), (21, 2), (22, 2), (23, 1), (24, 1), (25, 1), (26, 2), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 4), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 2), (44, 1), (45, 3), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1)]]


In [12]:
# Term frequency

[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('adaptation', 1),
  ('base', 1),
  ('become', 1),
  ('ca', 1),
  ('canadian', 1),
  ('carbon', 1),
  ('carbon_neutral', 2),
  ('change', 2),
  ('city', 1),
  ('climate', 2),
  ('combat', 1),
  ('community', 1),
  ('company', 1),
  ('conservation', 2),
  ('consultant', 1),
  ('design', 1),
  ('encourage', 1),
  ('environmental', 2),
  ('footprint', 1),
  ('funding', 1),
  ('future', 1),
  ('get', 2),
  ('glenn', 2),
  ('group', 1),
  ('guide', 1),
  ('individual', 1),
  ('information', 2),
  ('infrastructure', 1),
  ('livable', 1),
  ('mitigation', 1),
  ('municipality', 1),
  ('partnership', 1),
  ('peel', 1),
  ('project', 1),
  ('provide', 1),
  ('publication', 1),
  ('reduce', 1),
  ('region', 4),
  ('report', 1),
  ('reserve', 1),
  ('right', 1),
  ('series', 1),
  ('source', 1),
  ('sustainable', 2),
  ('toronto', 1),
  ('trca', 3),
  ('trusted', 1),
  ('university', 1),
  ('visit', 1),
  ('way', 1),
  ('website', 1),
  ('william', 1),
  ('write', 1),
  ('www', 1),
  ('york', 1

In [69]:
# Build LDA model

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=14, 
                                           random_state=42,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [77]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.074*"climate" + 0.062*"change" + 0.047*"canada" + 0.026*"report" + '
  '0.025*"ca" + 0.023*"adaptation" + 0.018*"canadian" + 0.015*"city" + '
  '0.015*"resource" + 0.014*"environment"'),
 (1,
  '0.046*"plan" + 0.046*"local" + 0.043*"government" + 0.042*"community" + '
  '0.031*"development" + 0.031*"planning" + 0.024*"land" + 0.023*"management" '
  '+ 0.020*"use" + 0.016*"www"'),
 (2,
  '0.061*"energy" + 0.049*"emission" + 0.048*"carbon" + 0.040*"greenhouse_gas" '
  '+ 0.032*"mitigation" + 0.029*"world" + 0.025*"atmosphere" + 0.022*"reduce" '
  '+ 0.020*"global" + 0.016*"gas"'),
 (3,
  '0.024*"change" + 0.021*"climate" + 0.015*"risk" + 0.011*"use" + '
  '0.011*"information" + 0.010*"need" + 0.009*"impact" + 0.008*"provide" + '
  '0.008*"process" + 0.008*"system"'),
 (4,
  '0.080*"season" + 0.046*"pest" + 0.030*"applicable" + 0.019*"mill" + '
  '0.017*"rat" + 0.016*"rating" + 0.014*"reasonable" + 0.014*"injury" + '
  '0.012*"wood" + 0.012*"edit"'),
 (5,
  '0.154*"coastal" + 0.

In [15]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=words_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -9.937024600441921

Coherence Score:  0.531039277528115


In [83]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
3      28.512276        1       1 -0.243159  0.006965
1      12.935347        1       2 -0.186712 -0.043610
0      12.406785        1       3 -0.208782  0.219052
7      10.489544        1       4 -0.174642  0.030726
11      9.010400        1       5 -0.123721 -0.271804
6       8.333980        1       6 -0.131179 -0.159151
9       7.939661        1       7 -0.229020  0.279416
10      4.317637        1       8 -0.008964 -0.109832
5       3.687510        1       9  0.019130 -0.167230
2       1.475717        1      10  0.166489  0.035228
12      0.315913        1      11  0.278754  0.044113
4       0.310357        1      12  0.279932  0.047291
13      0.241787        1      13  0.279633  0.046522
8       0.023090        1      14  0.282240  0.042315, topic_info=      Category          Freq            Term         Total  loglift  logprob
term                                                                        
9      Default  14688.000000         climate  14688.000000  30.0000  30.0000
7      Default  14112.000000          change  14112.000000  29.0000  29.0000
0      Default   5835.000000      adaptation   5835.000000  28.0000  28.0000
2200   Default   3194.000000         coastal   3194.000000  27.0000  27.0000
145    Default   4655.000000           water   4655.000000  26.0000  26.0000
2289   Default   2557.000000             sea   2557.000000  25.0000  25.0000
228    Default   4444.000000        increase   4444.000000  24.0000  24.0000
249    Default   4030.000000           local   4030.000000  23.0000  23.0000
272    Default   3408.000000            plan   3408.000000  22.0000  22.0000
216    Default   3789.000000      government   3789.000000  21.0000  21.0000
27     Default   2273.000000  infrastructure   2273.000000  20.0000  20.0000
579    Default   3965.000000          canada   3965.000000  19.0000  19.0000
208    Default   2551.000000          forest   2551.000000  18.0000  18.0000
11     Default   3988.000000       community   3988.000000  17.0000  17.0000
1252   Default   2583.000000           flood   2583.000000  16.0000  16.0000
8      Default   2408.000000            city   2408.000000  15.0000  15.0000
2284   Default   1404.000000      level_rise   1404.000000  14.0000  14.0000
225    Default   4736.000000          impact   4736.000000  13.0000  13.0000
246    Default   2288.000000           level   2288.000000  12.0000  12.0000
159    Default   4098.000000            area   4098.000000  11.0000  11.0000
411    Default   2496.000000            land   2496.000000  10.0000  10.0000
120    Default   2288.000000        planning   2288.000000   9.0000   9.0000
416    Default   3064.000000      management   3064.000000   8.0000   8.0000
641    Default   2583.000000     development   2583.000000   7.0000   7.0000
804    Default   1474.000000   vulnerability   1474.000000   6.0000   6.0000
143    Default   4076.000000             use   4076.000000   5.0000   5.0000
142    Default   1523.000000           urban   1523.000000   4.0000   4.0000
3      Default   1636.000000              ca   1636.000000   3.0000   3.0000
36     Default   1972.000000          reduce   1972.000000   2.0000   2.0000
325    Default   1477.000000     temperature   1477.000000   1.0000   1.0000
...        ...           ...             ...           ...      ...      ...
18360  Topic14      0.005528        entrench      0.971991   3.2040  -9.9940
4534   Topic14      0.005528     translation      0.972072   3.2040  -9.9939
13967  Topic14      0.005528         allowed      0.971992   3.2040  -9.9940
18657  Topic14      0.005527         cheaper      0.971777   3.2040  -9.9942
18658  Topic14      0.005527          cuckoo      0.971777   3.2040  -9.9942
18661  Topic14      0.005527            hair      0.971777   3.2040  -9.9942
18664  Topic14      0.005527        reinvest      0.971777   3.2040  -9.9942
18665  Topic1

LDA Model with TF-IDF

In [60]:
# TF-IDF

from gensim import corpora, models
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

In [62]:
lda_model_tfidf = gensim.models.ldamodel.LdaModel(corpus=corpus_tfidf,
                                           id2word=id2word,
                                           num_topics=14, 
                                           random_state=42,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [63]:
pprint(lda_model_tfidf.print_topics())
doc_lda_tfidf = lda_model_tfidf[corpus]

[(0,
  '0.015*"tree" + 0.009*"green" + 0.008*"fire" + 0.008*"heat" + '
  '0.007*"transportation" + 0.007*"rank" + 0.007*"challenge" + 0.006*"road" + '
  '0.006*"term" + 0.006*"carbon"'),
 (1,
  '0.000*"tschaplinski" + 0.000*"stinchcombe" + 0.000*"curran" + 0.000*"cier" '
  '+ 0.000*"wtr" + 0.000*"wld" + 0.000*"westland" + 0.000*"waterdsm" + '
  '0.000*"docintro" + 0.000*"toc"'),
 (2,
  '0.008*"maintenance" + 0.007*"goal" + 0.006*"iclei" + 0.005*"page" + '
  '0.005*"locate" + 0.005*"industry" + 0.005*"shore" + 0.004*"concern" + '
  '0.004*"product" + 0.004*"relative"'),
 (3,
  '0.012*"forest" + 0.008*"climate" + 0.008*"change" + 0.008*"adaptation" + '
  '0.008*"local" + 0.007*"government" + 0.007*"sfm" + 0.007*"increase" + '
  '0.006*"impact" + 0.006*"management"'),
 (4,
  '0.009*"org" + 0.007*"reference" + 0.006*"economy" + 0.006*"investment" + '
  '0.006*"hot" + 0.005*"move" + 0.005*"september" + 0.005*"disease" + '
  '0.005*"principle" + 0.005*"april"'),
 (5,
  '0.013*"hazard" + 0.01

In [64]:
# Compute Perplexity
print('\nPerplexity: ', lda_model_tfidf.log_perplexity(corpus))

# Compute Coherence Score
coherence_model_lda_tfidf = CoherenceModel(model=lda_model_tfidf, texts=words_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda_tfidf = coherence_model_lda_tfidf.get_coherence()
print('\nCoherence Score: ', coherence_lda_tfidf)


Perplexity:  -10.660302719923493

Coherence Score:  0.1840990036108526


In [65]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=4, step=2):

    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics, 
                                           random_state=42,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [66]:
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=words_lemmatized, start=2, limit=30, step=2)

In [67]:
limit=30; start=2; step=2;
x = range(start, limit, step)
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

Num Topics = 2  has Coherence Value of 0.3878
Num Topics = 4  has Coherence Value of 0.4778
Num Topics = 6  has Coherence Value of 0.4946
Num Topics = 8  has Coherence Value of 0.4808
Num Topics = 10  has Coherence Value of 0.4688
Num Topics = 12  has Coherence Value of 0.5082
Num Topics = 14  has Coherence Value of 0.531
Num Topics = 16  has Coherence Value of 0.5239
Num Topics = 18  has Coherence Value of 0.4841
Num Topics = 20  has Coherence Value of 0.52
Num Topics = 22  has Coherence Value of 0.4523
Num Topics = 24  has Coherence Value of 0.4515
Num Topics = 26  has Coherence Value of 0.4184
Num Topics = 28  has Coherence Value of 0.4355


Dominant Topic per Text

In [39]:
def format_topics_sentences(ldamodel=None, corpus=corpus, texts=df['Text']):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

In [40]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=df['Text'])

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,1.0,0.2709,"plan, local, government, community, developmen...",Toronto and Region Conservation TRCA has beco...
1,1,3.0,0.2346,"change, climate, risk, use, information, need,...",An Overview of Getting to Carbon Neutral: A Gu...
2,2,3.0,0.5257,"change, climate, risk, use, information, need,...",Getting to Carbon Neutral does not dictate wh...
3,3,3.0,0.3070,"change, climate, risk, use, information, need,...",Tens steps to carbon neutrality Getting to Ca...
4,4,2.0,0.2705,"energy, emission, carbon, greenhouse_gas, miti...",fired generating stations emit large amounts ...
5,5,10.0,0.4097,"infrastructure, city, urban, transportation, r...",GHG Reduction Strategies Sector Description of...
6,6,10.0,0.2499,"infrastructure, city, urban, transportation, r...",Sector Description of Approach Technological ...
7,7,3.0,0.3051,"change, climate, risk, use, information, need,...",Sector Description of Approach Technological ...
8,8,3.0,0.4427,"change, climate, risk, use, information, need,...",A climate change action strategy need not be ...
9,9,9.0,0.4093,"climate, adaptation, change, forest, impact, v...",EXECUTIVE SUMMARY Ontario s climate is warmin...
